In [2]:
from manim import *
import pandas as pd
import jupyter_capture_output

video_scene = " -v WARNING --disable_caching writing_visualization_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s writing_visualization_Scene"

Jupyter Capture Output v0.0.11


In [312]:
# data processing

# total writing data
writing_data = pd.read_csv(f"../external_media/Raw_Sample_Data_Report_Plus_Segs_Report_ERSERO0_segmentiert0.txt", delimiter = "\t", skiprows = 0, skipinitialspace = True).dropna(subset = "seg_path")
# reduce writing data to kinetic energy
kin_energy_writing_data = writing_data.loc[writing_data["seg_path"].str.contains('ERSERO0->kinEnergie')]
# add an integer time to the dataframe
kin_energy_writing_data["integer_time"] = kin_energy_writing_data["time"]*1000
kin_energy_writing_data["integer_time"] = kin_energy_writing_data["integer_time"].astype(int) 

# pause data
pause_data = pd.read_csv(f"../external_media/ERSERO0_kinEnergie.csv", delimiter = ",", skiprows = 0, skipinitialspace = True)
pause_data
kin_energy_writing_data

/tmp/ipykernel_192651/195029671.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kin_energy_writing_data["integer_time"] = kin_energy_writing_data["time"]*1000
/tmp/ipykernel_192651/195029671.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kin_energy_writing_data["integer_time"] = kin_energy_writing_data["integer_time"].astype(int)


,index,time,x,y,pressure,state,x_filtered,y_filtered,pressure_filtered,x_velocity,...,segment_id,status,series_id,run_id,stroke_id,stroke_type,seg_level,seg_path,seg_name,integer_time
5056,5056,206.940,9246,35384,112,24,9246.0,35384.0,112.0,1.256202,...,7,"['FIRST_PRESS', 'PRESSED']",10,68,83,0,2.0,ERSERO0->kinEnergie,E,206940
5057,5057,206.947,9246,35384,592,16,9246.0,35384.0,592.0,0.477689,...,7,['PRESSED'],10,68,83,0,2.0,ERSERO0->kinEnergie,E,206947
5058,5058,206.955,9246,35384,1088,16,9246.0,35384.0,1088.0,0.139569,...,7,['PRESSED'],10,68,83,0,2.0,ERSERO0->kinEnergie,E,206955
5059,5059,206.962,9246,35384,1616,16,9246.0,35384.0,1616.0,0.076590,...,7,['PRESSED'],10,68,83,0,2.0,ERSERO0->kinEnergie,E,206962
5060,5060,206.970,9246,35384,2128,16,9246.0,35384.0,2128.0,0.181402,...,7,['PRESSED'],10,68,83,0,2.0,ERSERO0->kinEnergie,E,206970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5778,5778,212.371,13756,34857,6707,16,13756.0,34857.0,6707.0,37.380578,...,16,['PRESSED'],10,78,91,0,3.0,ERSERO0->kinEnergie->v2,2,212371
5779,5779,212.379,13792,34857,6627,16,13792.0,34857.0,6627.0,33.837746,...,16,['PRESSED'],10,78,91,0,3.0,ERSERO0->kinEnergie->v2,2,212379
5780,5780,212.386,13827,34857,6338,16,13827.0,34857.0,6338.0,30.322885,...,16,['PRESSED'],10,78,91,0,3.0,ERSERO0->kinEnergie->v2,2,212386
5781,5781,212.394,13856,34857,4610,16,13856.0,34857.0,4610.0,26.814769,...,16,['PRESSED'],10,78,91,0,3.0,ERSERO0->kinEnergie->v2,2,212394


In [308]:
class SegPath(Mobject):
    def __init__(self, center, main_width, seg_data, **kwargs):
        super().__init__(**kwargs)
        self.center = center
        self.seg_data = seg_data

        # process writing data
        self.integer_time_data = seg_data["integer_time"]
        self.x_data = seg_data["x"]
        self.y_data = seg_data["y"]
        self.pressure_data = seg_data["pressure"]
        self.seg_name_data = seg_data["seg_name"]

        # set geometry of main frame
        x_data_range = max(self.x_data) - min(self.x_data)
        y_data_range = max(self.y_data) - min(self.y_data)
        print(x_data_range, y_data_range)

        self.main_width = main_width
        self.main_height = float(self.main_width) / x_data_range * y_data_range
        print(self.main_height, self.main_width)

        # main coordinate system
        self.main_npla = NumberPlane(
            x_range = (min(self.x_data)-x_data_range/10, max(self.x_data)+x_data_range/10, 100), y_range = (min(self.y_data)-y_data_range/2.5, max(self.y_data)+y_data_range/2.5, 100), x_length = self.main_width, y_length = self.main_height,
            x_axis_config = {"stroke_opacity": 0.215}, y_axis_config = {"stroke_opacity": 0.125}, background_line_style = {"stroke_opacity": 0.125}
        ).move_to(self.center)
        self.main_rect = Rectangle(width = self.main_width, height = self.main_height, stroke_width = 1, stroke_color = WHITE).move_to(self.center)
        self.main_square = Square(color = WHITE)
        # self.add(self.main_npla)
        self.add(self.main_rect)


    # returns the dot at position i
    def get_dot(self, int_time):
        if (int_time in self.integer_time_data.values):
            int_time_column = self.seg_data[self.seg_data["integer_time"] == int_time]
            pressure = int(int_time_column["pressure"].iloc[0])
            x_i = float(int_time_column["x"].iloc[0])
            y_i = float(int_time_column["y"].iloc[0])
            if pressure < 100:
                write_dot = Dot(radius = 0.025, color = RED, fill_opacity = 0.1, stroke_opacity = 0.1).move_to(self.main_npla.c2p(x_i, y_i, 0))
            else:
                write_dot = Dot(radius = 0.025, color = RED).move_to(self.main_npla.c2p(x_i, y_i, 0))
            return write_dot
            
            

class SegPathCoordinates(Mobject):
    def __init__(self, center, c_width, seg_data, **kwargs):
        super().__init__(**kwargs)
        self.center = center
        self.seg_data = seg_data

        # process writing data
        self.integer_time_data = seg_data["integer_time"]
        self.x_data = seg_data["x"]
        self.y_data = seg_data["y"]
        self.pressure_data = seg_data["pressure"]
        self.seg_name_data = seg_data["seg_name"]

        # geometry
        t_data_range = max(self.integer_time_data) - min(self.integer_time_data)
        x_data_range = max(self.x_data) - min(self.x_data)
        y_data_range = max(self.y_data) - min(self.y_data)
        self.c_width = c_width
        self.c_height = 2

        # x-coordinate system
        self.cx_npla = NumberPlane(
            x_range = (min(self.integer_time_data), max(self.integer_time_data), 1000), 
            y_range = (min(self.x_data)-x_data_range*2.5, max(self.x_data)+x_data_range*2.5, 100), x_length = self.c_width*0.8, y_length = self.c_height/2,
            x_axis_config = {"stroke_opacity": 0.215}, y_axis_config = {"stroke_opacity": 0.125}, background_line_style = {"stroke_opacity": 0.125}
        ).move_to(self.center + self.c_height/4*UP)

        # y-coordinate system
        self.cy_npla = NumberPlane(
            x_range = (min(self.integer_time_data), max(self.integer_time_data), 1000), 
            y_range = (min(self.y_data)-y_data_range*2.5, max(self.y_data)+y_data_range*2.5, 100), x_length = self.c_width*0.8, y_length = self.c_height/2,
            x_axis_config = {"stroke_opacity": 0.215}, y_axis_config = {"stroke_opacity": 0.125}, background_line_style = {"stroke_opacity": 0.125}
        ).move_to(self.center)

        # time number line
        self.number_line = NumberLine(x_range = (min(self.integer_time_data), max(self.integer_time_data), 1000), length = self.c_width*0.8).move_to(self.center - self.c_height/4*UP).add_tip(tip_width = 0.25, tip_length = 0.25)
        self.add(self.number_line)

        self.c_rect = Rectangle(width = self.c_width, height = self.c_height, stroke_width = 1, stroke_color = WHITE).move_to(self.center)
        x_descriptor = Tex(r"$x$", font_size = 28, color = WHITE).move_to(self.center + self.c_width/2*LEFT + self.c_height/4*UP + 0.375*RIGHT)
        y_descriptor = Tex(r"$y$", font_size = 28, color = WHITE).move_to(self.center + self.c_width/2*LEFT + 0.375*RIGHT)
        t_descriptor = Tex(r"$t$", font_size = 28, color = WHITE).next_to(self.number_line, RIGHT)
        # self.add(self.cx_npla, self.cy_npla)
        self.add(self.c_rect, x_descriptor, y_descriptor, t_descriptor)


    # x-dot method
    def get_xdot(self, int_time):
        if (int_time in self.integer_time_data.values):
            int_time_column = self.seg_data[self.seg_data["integer_time"] == int_time]
            pressure = int(int_time_column["pressure"].iloc[0])
            x_i = float(int_time_column["x"].iloc[0])
            if pressure < 100:
                write_dot = Dot(radius = 0.015, color = RED, fill_opacity = 0.1, stroke_opacity = 0.1).move_to(self.cx_npla.c2p(int_time, x_i, 0))
            else:
                write_dot = Dot(radius = 0.015, color = RED).move_to(self.cx_npla.c2p(int_time, x_i, 0))
            return write_dot
        

    # y-dot method
    def get_ydot(self, int_time):
        if (int_time in self.integer_time_data.values):
            int_time_column = self.seg_data[self.seg_data["integer_time"] == int_time]
            pressure = int(int_time_column["pressure"].iloc[0])
            y_i = float(int_time_column["y"].iloc[0])
            if pressure < 100:
                write_dot = Dot(radius = 0.015, color = RED, fill_opacity = 0.1, stroke_opacity = 0.1).move_to(self.cy_npla.c2p(int_time, y_i, 0))
            else:
                write_dot = Dot(radius = 0.015, color = RED).move_to(self.cy_npla.c2p(int_time, y_i, 0))
            return write_dot
        


class SegPathPauses(Mobject):
    def __init__(self, center, p_width, p_height, pause_data, **kwargs):
        super().__init__(**kwargs)
        self.center = center
        self.pause_data = pause_data

        self.names = pause_data["name"]
        self.pauses = pause_data["pauses"]

        names_ax_dict = dict(zip([i for i in range(len(self.names))], [*self.names])) 
        print(names_ax_dict)

        self.pause_ax = Axes(x_range = [0, len(self.names)-1, 1], y_range = [0, int(max(self.pauses))+1, 1], x_length = p_width, y_length = p_height, 
            tips = False
            ).move_to(self.center).scale(1.5).add_coordinates(names_ax_dict).scale(1/1.5)
        
        self.pauses_line_graph = self.pause_ax.plot_line_graph(
            x_values = [i for i in range(len(self.names))], y_values = [*self.pauses],
            line_color = BLUE,
            vertex_dot_style = dict(stroke_width = 1,  fill_color = BLUE),
            stroke_width = 1,
        )
        
        self.add(self.pause_ax, self.pauses_line_graph)

In [311]:
%%manim -qh --fps 60 $video_scene


class writing_visualization_Scene(Scene):
    def construct(self):

        # main path parameters
        main_path_center = np.array([-1.75, -1.5, 0])
        c_path_center = np.array([-1.75, 1.75, 0])
        pauses_path_center = np.array([4.5, 0, 0])
        
        main_path_width = 8
        pauses_height = 5
        pauses_width = 3

        seg_path = SegPath(main_path_center, main_path_width, kin_energy_writing_data)
        self.add(seg_path)

        seg_path_coordinates = SegPathCoordinates(c_path_center, main_path_width, kin_energy_writing_data)
        self.add(seg_path_coordinates)

        seg_path_pauses = SegPathPauses(pauses_path_center, pauses_width, pauses_height, pause_data)
        self.add(seg_path_pauses)

        min_time = min(kin_energy_writing_data["integer_time"])
        max_time = max(kin_energy_writing_data["integer_time"])
        self.wait(15)
        for int_time in range(min_time, max_time):
            dot = seg_path.get_dot(int_time)
            if dot:
                # print(int_time)
                self.add(dot)
            xdot = seg_path_coordinates.get_xdot(int_time)
            ydot = seg_path_coordinates.get_ydot(int_time)
            if xdot:
                self.add(xdot)
            if ydot:
                self.add(ydot)
            self.wait(1.0/60)
        self.wait(30)

Manim Community v0.18.1

4837 1550
2.563572462270002 8
{0: 'E', 1: '=', 2: '1', 3: '/', 4: '2', 5: 'm', 6: 'v', 7: '2'}


In [ ]:
# ffmpeg -i writing_visualization_Scene.mp4 -r 128 -vf "setpts=0.1*PTS" writing_visualization_Scene_10x.mp4